In [1]:
import os
import sys

SPARK_HOME = "/usr/lib/spark3"
PYSPARK_PYTHON = "/opt/conda/envs/dsenv/bin/python"
os.environ["PYSPARK_PYTHON"]= PYSPARK_PYTHON
os.environ["PYSPARK_DRIVER_PYTHON"]= PYSPARK_PYTHON
os.environ["SPARK_HOME"] = SPARK_HOME

PYSPARK_HOME = os.path.join(SPARK_HOME, "python/lib")
sys.path.insert(0, os.path.join(PYSPARK_HOME, "py4j-0.10.9.5-src.zip"))
sys.path.insert(0, os.path.join(PYSPARK_HOME, "pyspark.zip"))


from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set("spark.ui.port", "4321")

spark = SparkSession.builder.config(conf=conf).appName("dont kill me").getOrCreate()


from pyspark.sql.types import *

import pyspark.sql.functions as f

# spark.stop()

Picked up _JAVA_OPTIONS: 
Picked up _JAVA_OPTIONS: 
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark3/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/04/06 19:40:24 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


In [7]:
spark.stop()

In [3]:
train_path = '/datasets/amazon/train.json'
test_path = '/datasets/amazon/test83m.json'
# load sample of train path using spark:
schema = StructType(fields=[
    StructField("id", IntegerType()),
    StructField("overall", IntegerType()),
    StructField("vote", StringType()),
    StructField("verified", BooleanType()),
    StructField("reviewTime", StringType()),
    StructField("reviewerID", StringType()),
    StructField("asin", StringType()),
    StructField("reviewerName", StringType()),
    StructField("reviewText", StringType()),
    StructField("summary", StringType()),
    StructField("unixReviewTime", LongType()),
])
df_train = spark.read.json(train_path, schema=schema).sample(0.001)
df_test = spark.read.json(test_path, schema=schema).sample(0.0001)
# df.cache()

In [6]:
df.cache()

NameError: name 'df' is not defined

In [5]:
df_test.show(5)

+-----+-------+----+--------+-----------+--------------+----------+--------------+--------------------+--------------------+--------------+
|   id|overall|vote|verified| reviewTime|    reviewerID|      asin|  reviewerName|          reviewText|             summary|unixReviewTime|
+-----+-------+----+--------+-----------+--------------+----------+--------------+--------------------+--------------------+--------------+
| 1149|   null|  11|   false| 03 1, 2017| A8AE8LXG37B33|B017OJL1IE|   M. Anderson|Even though the A...|Works great on my...|    1488326400|
| 7876|   null|   6|   false| 12 1, 2016| A7E2C1E86ADAN|B01F9467IK|Matt Silverman|Love that it inte...|        Works great!|    1480550400|
|56676|   null|null|   false| 05 7, 2010|A3UJ9MJGFGHFNE|B000PKKAFK| Adamcommenter|Let me start by s...|Disappointing for...|    1273190400|
|58756|   null|null|    true|05 26, 2013| AZD3ON9ZMEGL6|B000URXP6E| huangweixiong|It smells good, s...|           i love it|    1369526400|
|59491|   null|null|

In [9]:
from pyspark.ml.feature import *
from pyspark.ml.regression import LinearRegression

Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:409)
	at java.net.ServerSocket.implAccept(ServerSocket.java:560)
	at java.net.ServerSocket.accept(ServerSocket.java:528)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


In [65]:
train = df_train.withColumn("text", f.concat(f.col("reviewerName"), f.lit(". "), f.col("reviewText"), f.lit(". "), f.col("summary"))).dropna(subset=["text"])
tokenizer = Tokenizer(inputCol="text", outputCol="words")
train = tokenizer.transform(train)
hasher = HashingTF(numFeatures=100, binary=True, inputCol=tokenizer.getOutputCol(), outputCol="features")
train = hasher.transform(train)
train = train.withColumnRenamed("overall", "label").select("id", "label", "features")

In [64]:
test = df_test.withColumn("text", f.concat(f.col("reviewerName"), f.lit(". "), f.col("reviewText"), f.lit(". "), f.col("summary"))).fillna(subset=["text"], value="")
test = tokenizer.transform(test)
test = hasher.transform(test)
test = test.withColumnRenamed("overall", "label").select("id","label", "features")

In [66]:
lr = LinearRegression(featuresCol="features", labelCol="label", regParam=0.3)
model = lr.fit(train)

In [67]:
predictions = model.transform(test.limit(10))
predictions = predictions.withColumn("prediction", f.when(f.col("prediction") > 5, 5).otherwise(f.when(f.col("prediction") < 1, 1).otherwise(f.col("prediction"))))

# predictions.filter((f.col("prediction") > 5) | (f.col("prediction") < 1)).count()

In [68]:
predictions.show(5)

+----+-----+--------------------+-----------------+
|  id|label|            features|       prediction|
+----+-----+--------------------+-----------------+
|   5|    1|(100,[5,17,26,27,...|4.281522231182959|
|4702|    5|(100,[3,5,6,9,12,...|4.121370756065989|
|5017|    1|(100,[0,7,17,19,2...|4.239557252331887|
|5442|    4|(100,[5,11,15,17,...|3.777200358720898|
|5640|    5|(100,[3,8,9,11,14...|4.447536747278852|
+----+-----+--------------------+-----------------+
only showing top 5 rows



In [52]:
from pyspark.ml import Pipeline

In [58]:
# from pyspark.ml import Pipeline, Transformer

# class ConcaterFillnaner_transformer(Transformer):
#   def __init__(self): super(ConcaterFillnaner_transformer, self).__init__()
#   def _transform(self, df):
#     df = df_test.withColumn("text", f.concat(f.col("reviewerName"), f.lit(". "), f.col("reviewText"), f.lit(". "), f.col("summary"))).fillna(subset=["text"], value="")
#     return df


# class ConcaterFillnaner_estimator(Transformer):
#   def __init__(self): super(ConcaterFillnaner_estimator, self).__init__()
#   def _fit(self, df):
#     df = df_train.withColumn("text", f.concat(f.col("reviewerName"), f.lit(". "), f.col("reviewText"), f.lit(". "), f.col("summary"))).dropna(subset=["text"])
#     return ConcaterFillnaner_transformer()


In [ ]:
pipeline = Pipeline(stages=[
  tokenizer,
  hasher,
  lr
])


In [60]:
pipeline.fit(df_train).transform(df_test).show(5)

TypeError: Cannot recognize a pipeline stage of type <class 'abc.ABCMeta'>.